In [ ]:
#EINLESEN DES DATAFRAMES UND VERARBEITUNG ZUM DATAFRAME MIT DEM ICH ARBEITE
import numpy as np
import pandas as pd
import csv
with open(r"C:\Users\roman\Desktop\DMS_data\DMS_data\GFP_AEQVI_Sarkisyan_2016.csv") as dms_GFP_datei:
    dms_GFP_datei_object = csv.reader(dms_GFP_datei, delimiter=',')
    print(dms_GFP_datei_object)
    for row in dms_GFP_datei_object:
        print(row)
original_datei = pd.read_csv(r"C:\Users\roman\Desktop\DMS_data\DMS_data\GFP_AEQVI_Sarkisyan_2016.csv")
print(original_datei)
#Einlesen

letzter_buchstabe = []
for index, row in original_datei.iterrows():
    last_character = row["mutant"][-1]
    letzter_buchstabe.append(last_character)
letzter_buchstabe_df = pd.DataFrame(letzter_buchstabe, columns=["New_AS"])
#print(letzter_buchstabe_df)
#Letzter Buchstabe in jeder Zeile

number_mutations = original_datei["mutant"].str.count(":") + 1
number_mutations_Single = number_mutations == 1
#print(number_mutations_Single)
#True sind alle Zeilen, die nur eine Mutation tragen

single_mutants_df = letzter_buchstabe_df[number_mutations_Single]
#print(single_mutants_df)
#Filtert alle True Zeilen aus beiden Dataframes und erstellt ein neues Dataframe. Er übernimmt automatisch die Filterung nac
dms_score_df = []
for index, row in original_datei.iterrows():
    dms_score = row["DMS_score"]
    dms_score_df.append(dms_score)
dms_score_df_alle = pd.DataFrame(dms_score_df, columns=["Fitness_Score"])
#print(dms_score_df_alle)
#erstellt Dataframe mit allen DMS-scores und der jeweiligen Experimentnummer
dms_score_filtered = dms_score_df_alle[number_mutations_Single]
#print(dms_score_filtered)
dms_score_filtered_newAS = single_mutants_df.join(dms_score_filtered)
#print(dms_score_filtered_newAS)
mutations_pos = []
for index, row in original_datei.iterrows():
    mutations_pos_nummer = row["mutant"][1:-1]
    mutations_pos.append(mutations_pos_nummer)
mutations_pos_df = pd.DataFrame(mutations_pos, columns=["Position"])
#print(mutations_pos_df)
#Entfernt erstmal nur das erste und letzte Zeichen
number_mutations = original_datei["mutant"].str.count(":") + 1
number_mutations_Single = number_mutations == 1
#print(number_mutations_Single)
#True sind alle Zeilen, die nur eine Mutation tragen

single_mutants_df_pos = mutations_pos_df[number_mutations_Single]
#print(single_mutants_df_pos)
#Erstellt ein Dataframe, dass die Positionen innerhalb des Proteins aller Einzelmutanten zeigt
mutations_pos_df_mit_scores = single_mutants_df_pos.join(dms_score_filtered)
#print(mutations_pos_df_mit_scores)
new_column = mutations_pos_df_mit_scores["Position"]
Roman_1 = dms_score_filtered_newAS.join(new_column)
Roman_1 = Roman_1[["Position", "New_AS", "Fitness_Score"]]
print(Roman_1)

In [ ]:
#TEST DES MANN-WHITNEY-U-TESTS MIT AUSGEDACHTER GRUPPE
group1 = [20, 23, 21, 25, 18, 17, 18, 24, 20, 24, 23, 19]
group2 = [24, 25, 21, 22, 23, 18, 17, 28, 24, 27, 21, 23]
import scipy.stats as stats

#Whitney-U-Test
stats.mannwhitneyu(group1, group2, alternative='two-sided')


In [ ]:
#MANN-WHITNEY-U-TEST MIT MEINEM DATAFRAME VOLLER SINGLEMUTANTEN
import pandas as pd
import scipy.stats as stats
#Ich will meine Werte sortieren
def position_sort(position):
    if position.isdigit():
        return int(position)
    else:
        return position
#Leere Liste
results = []
#Gruppierung basierend auf Position und Aminosäure
grouped_position = Roman_1.groupby('Position')
grouped_AS = Roman_1.groupby("New_AS")

# Konvertiert die Positionsspalte in numerische Werte
Roman_1['Position'] = pd.to_numeric(Roman_1['Position'])

#Whitney-Test für alle Kombinationen von Position und AS durchgeführt --> Signifikanter Unterschied in den Fitness-Scores zwischen verschiedenen Positionen und AS?
for position, group_position in grouped_position:
    for AS, group_AS in grouped_AS:
        fitness_scores_position = group_position['Fitness_Score'].values
        fitness_scores_AS = group_AS['Fitness_Score'].values

        statistic, p_value = stats.mannwhitneyu(fitness_scores_position, fitness_scores_AS, alternative='two-sided')
#Alpha-Wert selbst gesetzt
        alpha = 0.05

        result = {'Position': position,'Aminosäure': AS,'Teststatistik': statistic,'P-Wert': p_value,'Signifikanz': "Ja" if p_value < alpha else "Nein"}

        results.append(result)
#Ergebnisse ausspucken lassen
summary_df = pd.DataFrame(results)
#Sortieren nach aufsteigender Positionsnummer
summary_df = summary_df.sort_values('Position', ascending=True)
print(summary_df)

In [ ]:
#ERGEBNISSE DES MANN-WHITNEY-U-TESTS GEFILTERET: NUR DIE MUTATIONEN, DIE WIRKLICH IM DATAFRAME SIND.
# Überprüft, ob Position und Aminosäure in Roman_1 vorhanden sind
mask = summary_df.apply(lambda row: (row['Position'], row['Aminosäure']) in zip(Roman_1['Position'], Roman_1['New_AS']), axis=1)

# Filtert das summary_df entsprechend der Maske
summary_df_filtered = summary_df[mask]

print(summary_df_filtered)
#DATENFILTERUNG MUSS VORHER GESCHEHEN, UM DIE DATEN NICHT ZU MANIPULIEREN!

In [ ]:
#KRUSKAL-WALLIS-TEST FÜR POSITION GEGEN FITNESS-SCORE
#Der Kruskal-Wallis-Test ist eine nichtparametrische Methode, die auf Ränge basiert und daher keine Annahmen über die Verteilung der Daten macht. Er ermöglicht den Vergleich der Verteilungen zwischen mehreren Gruppen, unabhängig von ihrer Normalverteilung oder Varianzhomogenität.
import pandas as pd
import scipy.stats as stats

# Gruppen basierend auf Position erstellen
grouped_position_2 = Roman_1.groupby('Position')

# Liste zum Speichern der Fitness-Scores für jede Position
position_scores = []

# Schleife über die Positionen
for position, group_data in grouped_position_2:
    # Fitness-Scores für die aktuelle Position
    fitness_scores = group_data['Fitness_Score'].values
    position_scores.append(fitness_scores)

# Kruskal-Wallis-Test
statistic, p_value = stats.kruskal(*position_scores)

alpha = 0.05

print("Kruskal-Wallis-Test")
print(f"Teststatistik: {statistic}")
print(f"P-Wert: {p_value}")

if p_value < alpha:
    print("Es gibt einen signifikanten Unterschied zwischen den Positionen.")
else:
    print("Es gibt keinen signifikanten Unterschied zwischen den Positionen.")

In [ ]:
#KRUSKAL-WALLIS-TEST FÜR POSITION UND AMINOSÄURE GEGEN FITNESS-SCORE
import pandas as pd
import scipy.stats as stats

# Gruppen basierend auf Position und Aminosäure erstellen
grouped_position_as_2 = Roman_1.groupby(['Position', 'New_AS'])

# Liste zum Speichern der Fitness-Scores für jede Position und Aminosäure
position_as_scores = []

# Schleife über die Positionen und Aminosäuren
for (position, amino_acid), group_data in grouped_position_as_2:
    # Fitness-Scores für die aktuelle Position und Aminosäure
    fitness_scores = group_data['Fitness_Score'].values
    position_as_scores.append(fitness_scores)
#Die Fitness-Scores für jede Position und Aminosäure werden in einer Liste gespeichert und als Argumente für den Kruskal-Wallis-Test verwendet
# Kruskal-Wallis-Test
statistic, p_value = stats.kruskal(*position_as_scores)

alpha = 0.05

print("Kruskal-Wallis-Test")
print(f"Teststatistik: {statistic}")
print(f"P-Wert: {p_value}")

if p_value < alpha:
    print("Es gibt einen signifikanten Unterschied zwischen den Positionen und Aminosäuren.")
else:
    print("Es gibt keinen signifikanten Unterschied zwischen den Positionen und Aminosäuren.")

In [ ]:
#DANKE FÜR DEINE HILFE!